In [ ]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient('localhost', 27017) 
db = client.get_database('Project') 
df1 = pd.read_excel('E:\College\level 4 term 2\Big Data\project\excel\HeartRisk.xlsx') # change path of file
data1 = df1.to_dict(orient='records')
collection1 = db['HeartRisk']
collection1.insert_many(data1)
##############################################
df2 = pd.read_excel('E:\College\level 4 term 2\Big Data\project\excel\Patients.xlsx') # change path of file
data2 = df2.to_dict(orient='records')
collection2 = db['Patients']
collection2.insert_many(data2)
##############################################
df3 = pd.read_excel('E:\College\level 4 term 2\Big Data\project\excel\Lifestyle.xlsx') # change path of file
data3 = df3.to_dict(orient='records')
collection3 = db['Lifestyle']
collection3.insert_many(data3)
##############################################
df4 = pd.read_excel('E:\College\level 4 term 2\Big Data\project\excel\Health.xlsx') # change path of file
data4 = df4.to_dict(orient='records')
collection4 = db['Health']
collection4.insert_many(data4)

client.close()

In [ ]:
# This code is to ensure that the data was successfully saved from the files
from pymongo import MongoClient
client2 = MongoClient('localhost', 27017) 
# access a database
db = client2.get_database("Project")  

# access a collection
collection = db.get_collection("HeartRisk")

cursor = collection.find()

for document in cursor:
    print(document)

In [ ]:
from flask import Flask, request, jsonify
from pymongo import MongoClient
from bson import json_util

app = Flask(__name__)


# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['Project']

# Define collections
health_collection = db['Health']
heart_risk_collection = db['HeartRisk']
patients_collection = db['Patients']
lifestyle_collection = db['Lifestyle']

# CRUD operations for Health collection
@app.route('/health/many', methods=['POST'])
def create_health_records(): 
    data = request.json
    result = health_collection.insert_many(data)
    return jsonify({"message": "Health Records Created", "ids": [str(id) for id in result.inserted_ids]}), 201
##############################################
@app.route('/health', methods=['POST'])
def create_health_record():
    data = request.json
    result = health_collection.insert_one(data)
    return jsonify({"message": "Health Record Created", "id": str(result.inserted_id)}), 201
##############################################
@app.route('/health/getall', methods=['GET'])
def get_all_health_records():
    records = list(health_collection.find())
    # Convert ObjectId to string before serializing to JSON
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/health/<patient_id>', methods=['GET'])
def read_health_record(patient_id):
    record = health_collection.find_one({"Patient ID": patient_id})
    if record:
        serialized_record = json_util.dumps(record)
        return serialized_record, 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/health/<patient_id>', methods=['PUT'])
def update_health_record(patient_id):
    data = request.json
    result = health_collection.update_one({"Patient ID": patient_id}, {"$set": data})
    if result.modified_count:
        return jsonify({"message": "Record updated"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/health/<patient_id>', methods=['DELETE'])
def delete_health_record(patient_id):
    result = health_collection.delete_one({"Patient ID": patient_id})
    if result.deleted_count:
        return jsonify({"message": "Record deleted"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################    
    # Index API for Health collection
@app.route('/health/index/<field>/<string:direction>', methods=['GET'])
def create_health_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    # Check if field exists in any document
    if not health_collection.find_one({field: {"$exists": True}}):
        return jsonify({"message": f"Field '{field}' does not exist in any document"}), 400
    
    result = health_collection.create_index([(field, dir_value)])
    return jsonify({"message": "Index created", "index": result}), 201
##############################################
    # Hint API for Health collection
@app.route('/health/hint/<field>/<string:direction>', methods=['GET'])
def hint_health_collection(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{'1' if dir_value == 1 else '-1'}"
    indexes = health_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index on field '{field}' with direction '{direction}' does not exist"}), 400

    records = list(health_collection.find().hint([(field, dir_value)]))
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/health/index/<field>/<string:direction>', methods=['DELETE'])
def drop_health_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{dir_value}"
    indexes = health_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index '{index_name}' does not exist"}), 400

    health_collection.drop_index(index_name)
    return jsonify({"message": f"Index '{index_name}' dropped"}), 200
##############################################
@app.route('/health/count', methods=['GET'])
def count_diabetes_in_health():
    count = health_collection.count_documents({"Diabetes": 1})
    return jsonify({"The number of patients who have diabetes is ": count}), 200
##############################################
@app.route('/health/aggregate', methods=['GET'])
def aggregate_previous_heart_problems_in_health():
    pipeline = [
        {"$match": {"Previous Heart Problems": 1}},
        {"$sort": {"Heart Rate": -1}}
    ]
    results = list(health_collection.aggregate(pipeline))
    serialized_results = json_util.dumps(results)
    return serialized_results, 200


##############################################
# CRUD operations for Heart Risk collection
@app.route('/heartrisk/many', methods=['POST'])
def create_heart_risk_records(): 
    data = request.json
    result = heart_risk_collection.insert_many(data)
    return jsonify({"message": "Heart Risk Records Created", "ids": [str(id) for id in result.inserted_ids]}), 201
##############################################
@app.route('/heartrisk', methods=['POST'])
def create_heart_risk_record():
    data = request.json
    result = heart_risk_collection.insert_one(data)
    return jsonify({"message": "Heart Risk Record Created", "id": str(result.inserted_id)}), 201
##############################################
@app.route('/heartrisk/getall', methods=['GET'])
def get_all_heart_risk_records():
    records = list(heart_risk_collection.find())
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/heartrisk/<patient_id>', methods=['GET'])
def read_heart_risk_record(patient_id):
    record = heart_risk_collection.find_one({"Patient ID": patient_id})
    if record:
        serialized_record = json_util.dumps(record)
        return serialized_record, 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/heartrisk/<patient_id>', methods=['PUT'])
def update_heart_risk_record(patient_id):
    data = request.json
    result = heart_risk_collection.update_one({"Patient ID": patient_id}, {"$set": data})
    if result.modified_count:
        return jsonify({"message": "Record updated"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/heartrisk/<patient_id>', methods=['DELETE'])
def delete_heart_risk_record(patient_id):
    result = heart_risk_collection.delete_one({"Patient ID": patient_id})
    if result.deleted_count:
        return jsonify({"message": "Record deleted"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404    
##############################################    
    # Index API for Heart Risk collection
@app.route('/heartrisk/index/<field>/<string:direction>', methods=['GET'])
def create_heart_risk_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    # Check if field exists in any document
    if not heart_risk_collection.find_one({field: {"$exists": True}}):
        return jsonify({"message": f"Field '{field}' does not exist in any document"}), 400
    
    result = heart_risk_collection.create_index([(field, dir_value)])
    return jsonify({"message": "Index created", "index": result}), 201
##############################################
    # Hint API for Heart Risk collection
@app.route('/heartrisk/hint/<field>/<string:direction>', methods=['GET'])
def hint_heart_risk_collection(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{'1' if dir_value == 1 else '-1'}"
    indexes = heart_risk_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index on field '{field}' with direction '{direction}' does not exist"}), 400

    records = list(heart_risk_collection.find().hint([(field, dir_value)]))
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/heartrisk/index/<field>/<string:direction>', methods=['DELETE'])
def drop_heartrisk_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{dir_value}"
    indexes = heart_risk_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index '{index_name}' does not exist"}), 400

    heart_risk_collection.drop_index(index_name)
    return jsonify({"message": f"Index '{index_name}' dropped"}), 200
##############################################
@app.route('/heartrisk/count', methods=['GET'])
def count_heart_attack_risk():
    count = heart_risk_collection.count_documents({"Heart Attack Risk": 1})
    return jsonify({"Patients with Heart Attack Risk number is ": count}), 200
##############################################
@app.route('/heartrisk/aggregate', methods=['GET'])
def aggregate_heart_attack_risk():
    pipeline = [
        {"$match": {"Heart Attack Risk": 1}}
    ]
    results = list(heart_risk_collection.aggregate(pipeline))
    serialized_results = json_util.dumps(results)
    return serialized_results, 200


##############################################
# CRUD operations for Patients collection
@app.route('/patients/many', methods=['POST'])
def create_patient_records(): 
    data = request.json
    result = patients_collection.insert_many(data)
    return jsonify({"message": "Patient Records Created", "ids": [str(id) for id in result.inserted_ids]}), 201
##############################################
@app.route('/patients', methods=['POST'])
def create_patient_record():
    data = request.json
    result = patients_collection.insert_one(data)
    return jsonify({"message": "Patient Record Created", "id": str(result.inserted_id)}), 201
##############################################
@app.route('/patients/getall', methods=['GET'])
def get_all_patient_records():
    records = list(patients_collection.find())
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/patients/<patient_id>', methods=['GET'])
def read_patient_record(patient_id):
    record = patients_collection.find_one({"Patient ID": patient_id})
    if record:
        serialized_record = json_util.dumps(record)
        return serialized_record, 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/patients/<patient_id>', methods=['PUT'])
def update_patient_record(patient_id):
    data = request.json
    result = patients_collection.update_one({"Patient ID": patient_id}, {"$set": data})
    if result.modified_count:
        return jsonify({"message": "Record updated"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/patients/<patient_id>', methods=['DELETE'])
def delete_patient_record(patient_id):
    result = patients_collection.delete_one({"Patient ID": patient_id})
    if result.deleted_count:
        return jsonify({"message": "Record deleted"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404  
##############################################    
    # Index API for Patients collection
@app.route('/patients/index/<field>/<string:direction>', methods=['GET'])
def create_patients_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400
    
    # Check if field exists in any document
    if not patients_collection.find_one({field: {"$exists": True}}):
        return jsonify({"message": f"Field '{field}' does not exist in any document"}), 400
    
    result = patients_collection.create_index([(field, dir_value)])
    return jsonify({"message": "Index created", "index": result}), 201
##############################################
    # Hint API for Patients collection
@app.route('/patients/hint/<field>/<string:direction>', methods=['GET'])
def hint_patients_collection(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{'1' if dir_value == 1 else '-1'}"
    indexes = patients_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index on field '{field}' with direction '{direction}' does not exist"}), 400

    records = list(patients_collection.find().hint([(field, dir_value)]))
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/patients/index/<field>/<string:direction>', methods=['DELETE'])
def drop_patients_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{dir_value}"
    indexes = patients_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index '{index_name}' does not exist"}), 400

    patients_collection.drop_index(index_name)
    return jsonify({"message": f"Index '{index_name}' dropped"}), 200
##############################################
@app.route('/patients/count', methods=['GET'])
def count_age_over_60_in_patients():
    count = patients_collection.count_documents({"Age": {"$gt": 60}})
    return jsonify({"The number of patients ages greater than 60 is ": count}), 200
##############################################
@app.route('/patients/aggregate', methods=['GET'])
def aggregate_africa_male_in_patients():
    pipeline = [
        {"$match": {"Continent": "Africa", "Sex": "Male"}},
        {"$sort": {"Income": 1}}
    ]
    results = list(patients_collection.aggregate(pipeline))
    serialized_results = json_util.dumps(results)
    return serialized_results, 200


##############################################
# CRUD operations for Lifestyle collection
@app.route('/lifestyle/many', methods=['POST'])
def create_lifestyle_records(): 
    data = request.json
    result = lifestyle_collection.insert_many(data)
    return jsonify({"message": "Lifestyle Records Created", "ids": [str(id) for id in result.inserted_ids]}), 201
##############################################
@app.route('/lifestyle', methods=['POST'])
def create_lifestyle_record():
    data = request.json
    result = lifestyle_collection.insert_one(data)
    return jsonify({"message": "Lifestyle Record Created", "id": str(result.inserted_id)}), 201
##############################################
@app.route('/lifestyle/getall', methods=['GET'])
def get_all_lifestyle_records():
    records = list(lifestyle_collection.find())
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/lifestyle/<patient_id>', methods=['GET'])
def read_lifestyle_record(patient_id):
    record = lifestyle_collection.find_one({"Patient ID": patient_id})
    if record:
        serialized_record = json_util.dumps(record)
        return serialized_record, 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/lifestyle/<patient_id>', methods=['PUT'])
def update_lifestyle_record(patient_id):
    data = request.json
    result = lifestyle_collection.update_one({"Patient ID": patient_id}, {"$set": data})
    if result.modified_count:
        return jsonify({"message": "Record updated"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404
##############################################
@app.route('/lifestyle/<patient_id>', methods=['DELETE'])
def delete_lifestyle_record(patient_id):
    result = lifestyle_collection.delete_one({"Patient ID": patient_id})
    if result.deleted_count:
        return jsonify({"message": "Record deleted"}), 200
    else:
        return jsonify({"message": "Record not found"}), 404   
##############################################    
    # Index API for Lifestyle collection
@app.route('/lifestyle/index/<field>/<string:direction>', methods=['GET'])
def create_lifestyle_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400
    
    # Check if field exists in any document
    if not lifestyle_collection.find_one({field: {"$exists": True}}):
        return jsonify({"message": f"Field '{field}' does not exist in any document"}), 400

    result = lifestyle_collection.create_index([(field, dir_value)])
    return jsonify({"message": "Index created", "index": result}), 201
##############################################
    # Hint API for Lifestyle collection
@app.route('/lifestyle/hint/<field>/<string:direction>', methods=['GET'])
def hint_lifestyle_collection(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{'1' if dir_value == 1 else '-1'}"
    indexes = lifestyle_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index on field '{field}' with direction '{direction}' does not exist"}), 400
    records = list(lifestyle_collection.find().hint([(field, dir_value)]))
    serialized_records = json_util.dumps(records)
    return serialized_records, 200
##############################################
@app.route('/lifestyle/index/<field>/<string:direction>', methods=['DELETE'])
def drop_lifestyle_index(field, direction):
    if direction == '1':
        dir_value = 1
    elif direction == '-1':
        dir_value = -1
    else:
        return jsonify({"message": "Direction must be '1' (ascending) or '-1' (descending)"}), 400

    index_name = f"{field}_{dir_value}"
    indexes = lifestyle_collection.index_information()
    if index_name not in indexes:
        return jsonify({"message": f"Index '{index_name}' does not exist"}), 400

    lifestyle_collection.drop_index(index_name)
    return jsonify({"message": f"Index '{index_name}' dropped"}), 200
##############################################
@app.route('/lifestyle/count', methods=['GET'])
def count_smoking_and_alcohol_in_lifestyle():
    count = lifestyle_collection.count_documents({"Smoking": 1, "Alcohol Consumption": 1})
    return jsonify({"Number of patients who smoke and consume alcohol is ": count}), 200
##############################################
@app.route('/lifestyle/aggregate', methods=['GET'])
def aggregate_unhealthy_diet_in_lifestyle():
    pipeline = [
        {"$match": {"Diet": "Unhealthy"}},
        {"$sort": {"Exercise Hours Per Week": 1}}
    ]
    results = list(lifestyle_collection.aggregate(pipeline))
    serialized_results = json_util.dumps(results)
    return serialized_results, 200




################################################################################################



@app.route('/health/mapreduce/average_stress_level', methods=['GET'])
def mapreduce_average_stress_level():
    map_function = """
    function() {
        if (this.Diabetes == 1) {
            emit('average of stress level to patients who have diabetes', this['Stress Level']);
        }
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.avg(values);
    }
    """
    res = db.command({
        "mapReduce": "Health",
        "map": map_function,
        "reduce": reduce_function,
        "out": "average_stress_level"
    })
    resultt = db.average_stress_level.find()
    result_list = list(resultt)
    return json_util.dumps(result_list), 200
##############################################
@app.route('/health/mapreduce/high_cholesterol_count', methods=['GET'])
def mapreduce_high_cholesterol_count():
    map_function = """
    function() {
        if (this.Cholesterol > 200) {
            emit('Number of patients have high cholesterol', 1);
        }
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.sum(values);
    }
    """
    res = db.command({
        "mapReduce": "Health",
        "map": map_function,
        "reduce": reduce_function,
        "out": "high_cholesterol_count"
    })
    resultt = db.high_cholesterol_count.find()
    result_list = list(resultt)
    return json_util.dumps(result_list), 200



##############################################
@app.route('/heartrisk/mapreduce/high_risk_count', methods=['GET'])
def mapreduce_high_risk_count():
    map_function = """
    function() {
        if (this['Heart Attack Risk'] == 0) {
            emit('Number of patients not have Heart Attack Risk', 1);
        }
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.sum(values);
    }
    """
    res = db.command({
        "mapReduce": "HeartRisk",
        "map": map_function,
        "reduce": reduce_function,
        "out": "high_risk_count"
    })
    resultt = db.high_risk_count.find()
    result_list = list(resultt)
    return json_util.dumps(result_list), 200



##############################################
@app.route('/patients/mapreduce/average_age_per_continent', methods=['GET'])
def mapreduce_average_age_per_continent():
    map_function = """
    function() {
        emit(this.Continent, this.Age);
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.avg(values);
    }
    """
    res = db.command({
        "mapReduce": "Patients",
        "map": map_function,
        "reduce": reduce_function,
        "out": "average_age_per_continent"
    })
    resultt = db.average_age_per_continent.find()
    result_list = list(resultt)
    return json_util.dumps(result_list), 200
##############################################
@app.route('/patients/mapreduce/big_country_big_avg_income', methods=['GET'])
def mapreduce_big_country_big_avg_income():
    map_function = """
    function() {
        emit(this.Country, this.Income);
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.avg(values);
    }
    """
    res = db.command({
        "mapReduce": "Patients",
        "map": map_function,
        "reduce": reduce_function,
        "out": "big_country_big_avg_income"
    })

    max_avg = db.big_country_big_avg_income.find_one(sort=[("value", -1)])
    return json_util.dumps(("The country with the highest average income", max_avg)), 200



##############################################
@app.route('/lifestyle/mapreduce/count_alcohol_non_smokers', methods=['GET'])
def mapreduce_average_alcohol_non_smokers():
    map_function = """
    function() {
        if (this.Smoking == 0 && this['Alcohol Consumption'] == 1) {
            emit('count alcohol without smoking', 1);
        }
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.sum(values);
    }
    """
    res = db.command({
        "mapReduce": "Lifestyle",
        "map": map_function,
        "reduce": reduce_function,
        "out": "alcohol_avg_non_smokers"
    })
    resultt = db.alcohol_avg_non_smokers.find()
    result_list = list(resultt)
    return json_util.dumps(result_list), 200
##############################################
@app.route('/lifestyle/mapreduce/average_sleep_smokers', methods=['GET'])
def mapreduce_average_sleep_smokers():
    map_function = """
    function() {
        if (this.Smoking == 1 && !isNaN(this['Sleep Hours Per Day'])) {
            emit('Average number of hours of sleep for patients who smoke', this['Sleep Hours Per Day']);
        }
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.avg(values);
    }
    """
    res = db.command({
        "mapReduce": "Lifestyle",
        "map": map_function,
        "reduce": reduce_function,
        "out": "average_sleep_smokers"
    })
    resultt = db.average_sleep_smokers.find()
    result_list = list(resultt)
    return json_util.dumps(result_list), 200
##############################################
@app.route('/lifestyle/mapreduce/low_exercise_count', methods=['GET'])
def mapreduce_low_exercise_count():
    map_function = """
    function() {
        if (this['Exercise Hours Per Week'] < 2) {
            emit('low exercise hours per week count', 1);
        }
    }
    """
    reduce_function = """
    function(key, values) {
        return Array.sum(values);
    }
    """
    res=db.command({
    "mapReduce":"Lifestyle",
    "map":map_function,
    "reduce":reduce_function,
    "out":"Output_sum"
    })
    resultt=db.Output_sum.find()

    result_list = list(resultt)
    return json_util.dumps(result_list), 200



if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
